In [1]:
import tensorflow as tf
from tensorflow import keras
import os
tf.__version__

'2.0.0-alpha0'

In [2]:
tf.config.gpu.set_per_process_memory_growth(True)

In [3]:
(img_trn, label_trn), (img_tst, label_tst) = keras.datasets.mnist.load_data()

In [4]:
label_trn.shape

(60000,)

In [5]:
num_pixel = 28*28
img_trn = img_trn.reshape(-1, num_pixel)/255.0
img_tst = img_tst.reshape(-1, num_pixel)/255.0

In [6]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(num_pixel,)),
        keras.layers.Dropout(0.20),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

In [9]:
history = model.fit(img_trn, label_trn, epochs=10, validation_data = (img_tst, label_tst), callbacks=[cp_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
59712/60000 [============================>.] - ETA: 0s - loss: 0.2187 - accuracy: 0.9365
Epoch 00001: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 8s 139us/sample - loss: 0.2183 - accuracy: 0.9367 - val_loss: 0.1082 - val_accuracy: 0.9670
Epoch 2/10
59808/60000 [============================>.] - ETA: 0s - loss: 0.0967 - accuracy: 0.9705
Epoch 00002: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0966 - accuracy: 0.9706 - val_loss: 0.0815 - val_accuracy: 0.9748
Epoch 3/10
59872/60000 [============================>.] - ETA: 0s - loss: 0.0698 - accuracy: 0.9777
Epoch 00003: saving model to training_1/cp.ckpt
60000/60000 [==============================] - 8s 129us/sample - loss: 0.0697 - accuracy: 0.9777 - val_loss: 0.0804 - val_accuracy: 0.9741
Epoch 4/10
59584/60000 [============================>.] - ETA: 0s - loss: 0.0520 - accura

In [10]:
!ls {checkpoint_dir}

checkpoint
cp.ckpt.data-00000-of-00001
cp.ckpt.index


In [11]:
model = create_model()
loss, acc = model.evaluate(img_tst, label_tst)
print(f'Untrained model; accuracy: {acc*100:5.2f}%')

10000/10000 [==============================] - 1s 79us/sample - loss: 2.4203 - accuracy: 0.0926
Untrained model; accuracy:  9.26%


In [12]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(img_tst, label_tst)
print(f'Restored model; accuracy: {acc*100:5.2f}%')

10000/10000 [==============================] - 1s 73us/sample - loss: 0.0769 - accuracy: 0.9824
Restored model; accuracy: 98.24%


In [17]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True, period=5)

In [18]:
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(img_trn, label_trn, epochs=50, callbacks=[cp_callback], validation_data = (img_tst, label_tst), verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [19]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [20]:
model.save_weights('./checkpoints/my_checkpoint')
model.load_weights('./checkpoints/my_checkpoint')
loss, acc = model.evaluate(img_tst, label_tst)
print(f'Restored model; accuracy: {acc*100:5.2f}%')

10000/10000 [==============================] - 1s 71us/sample - loss: 0.1439 - accuracy: 0.9832
Restored model; accuracy: 98.32%


In [21]:
model = create_model()
model.load_weights(latest)
model.save('my_model.h5')

In [22]:
model2 = keras.models.load_model('my_model.h5')

W0607 14:08:34.692910 26584 hdf5_format.py:261] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


In [23]:
loss, acc = model2.evaluate(img_tst, label_tst)
print(f'Restored model; accuracy: {acc*100:5.2f}%')

10000/10000 [==============================] - 1s 75us/sample - loss: 0.1439 - accuracy: 0.9832
Restored model; accuracy: 98.32%
